In [3]:
import numpy as np
import pandas as pd

# The the constants
phases = ('Quartz', 'Feldspar', 'Dolomite', 'Calcite')

f_Quartz = np.array([0.45])
f_Feldspar = np.array([0.2])
f_Dolomite = np.array([0.05])
f_Calcite = np.array([0.1])


f_solid_components = np.array([f_Quartz, f_Feldspar, f_Dolomite, f_Calcite]).T

porosity = np.linspace(0, 1, 100)
porosity_values = np.round(porosity, 2)

initial_porosity = 1-(np.sum(f_solid_components, axis=1))
print(f"You initial porosity is: {round(float(initial_porosity),2)}")

# Calculate solid fractions for each porosity value
f_Solid_interpolated = np.zeros((np.shape(porosity_values)[0], np.shape(f_solid_components)[1]))

# Find the index corresponding to the initial_porosity
initial_index = int(initial_porosity * len(f_Solid_interpolated))

f_Solid_interpolated[initial_index, :] = f_solid_components

# Linear interpolation based on porosity for the rest of the rows
for index in range(len(porosity_values)):   
    porosity_tmp = np.round(porosity_values[index],2)        
    f_Solid_interpolated[index, :] = (1 - porosity_tmp) * f_solid_components / np.sum(f_solid_components)

# Add summation along axis=1 as a new column
row_sums = np.sum(f_Solid_interpolated, axis=1, keepdims=True)
f_Solid_interpolated_check = np.hstack((f_Solid_interpolated, 1-row_sums))

# # Check if the sum of the normalized solid fractions is equal to 1
column_sums = np.sum(f_Solid_interpolated_check, axis=1)
if not np.allclose(column_sums, 1):
    problematic_columns = np.where(~np.isclose(column_sums, 1))[0]
    raise ValueError(f'The sum of each column + porosity must be approximately equal to 1. Problematic columns: {problematic_columns}.')

# Create column names based on the length and strings of components
column_names = [component for component in phases]
df_result = pd.DataFrame(f_Solid_interpolated, columns=column_names)
# Add "Porosity" column
df_result['Porosity'] = porosity

print(df_result)

You initial porosity is: 0.2
      Quartz  Feldspar  Dolomite  Calcite  Porosity
0   0.562500    0.2500  0.062500  0.12500  0.000000
1   0.556875    0.2475  0.061875  0.12375  0.010101
2   0.551250    0.2450  0.061250  0.12250  0.020202
3   0.545625    0.2425  0.060625  0.12125  0.030303
4   0.540000    0.2400  0.060000  0.12000  0.040404
..       ...       ...       ...      ...       ...
95  0.022500    0.0100  0.002500  0.00500  0.959596
96  0.016875    0.0075  0.001875  0.00375  0.969697
97  0.011250    0.0050  0.001250  0.00250  0.979798
98  0.005625    0.0025  0.000625  0.00125  0.989899
99  0.000000    0.0000  0.000000  0.00000  1.000000

[100 rows x 5 columns]
